In [ ]:
import pandas as pd
import numpy as np

# Load all sheets from the Excel file
file_path = "copy_whole_forward.xlsx"  # Change if needed
xls = pd.read_excel(file_path, sheet_name=None)  # Load all sheets as dict

# Function to clean each sheet
def clean_sheet(df):
    # 1. Remove rows with missing START or END
    df = df.dropna(subset=["START", "END"])

    # 2. Convert DISTANCE to meters
    def convert_distance(dist):
        if isinstance(dist, str):
            dist = dist.strip()
            if "km" in dist:
                return float(dist.replace(" km", "").strip()) * 1000
            elif "m" in dist:
                return float(dist.replace(" m", "").strip())
        return np.nan

    df["DISTANCE_M"] = df["DISTANCE"].apply(convert_distance)

    # 3. Convert durations to seconds
    def convert_duration(duration):
        if isinstance(duration, str):
            duration = duration.strip().lower()
            if "hour" in duration:
                parts = duration.replace("hours", "").replace("hour", "").replace("mins", "").replace("min", "").split()
                if len(parts) == 2:
                    return int(parts[0]) * 3600 + int(parts[1]) * 60
                elif len(parts) == 1:
                    return int(parts[0]) * 3600
            elif "min" in duration:
                return int(duration.replace("mins", "").replace("min", "").strip()) * 60
            elif "sec" in duration:
                return int(duration.replace("secs", "").replace("sec", "").strip())
        return np.nan

    df["DURATION_SEC"] = df["DURATION"].apply(convert_duration)
    df["DURATION_IN_TRAFFIC_SEC"] = df["DURATION_IN_TRAFFIC"].apply(convert_duration)

    # 4. Remove duplicates
    df = df.drop_duplicates()

    # 5. Create time-based columns
    df["DATE"] = df["DATE&TIME"].dt.date
    df["HOUR"] = df["DATE&TIME"].dt.hour
    df["WEEKDAY"] = df["DATE&TIME"].dt.day_name()

    return df

# Clean all sheets
cleaned_sheets = {sheet_name: clean_sheet(df.copy()) for sheet_name, df in xls.items()}

# Save the cleaned data back to a new Excel file
output_path = "cleaned_all_sheets_forward.xlsx"
with pd.ExcelWriter(output_path) as writer:
    for sheet_name, df in cleaned_sheets.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"✅ All sheets cleaned and saved to: {output_path}")


In [ ]:
import pandas as pd

file_path = "copy_whole_forward.xlsx"  # Replace with your path
xls = pd.read_excel(file_path, sheet_name=None)

missing_start_end = []

for sheet_name, df in xls.items():
    if not {"START", "END"}.issubset(df.columns):
        missing_start_end.append(sheet_name)

print("Sheets missing 'START' and 'END':")
print(missing_start_end)


In [ ]:
import pandas as pd
import re

# Load the Excel file
file_path = "copy_whole_forward.xlsx"
xls = pd.ExcelFile(file_path)
cleaned_sheets = {}

# Conversion helpers
def convert_distance(dist):
    if isinstance(dist, str):
        if "km" in dist:
            num = float(dist.replace("km", "").strip())
            return int(num * 1000)
        elif "m" in dist:
            return int(dist.replace("m", "").strip())
    return pd.NA

def convert_duration(dur):
    if isinstance(dur, str):
        minutes = re.findall(r"(\d+)\s*min", dur)
        seconds = re.findall(r"(\d+)\s*sec", dur)
        total = 0
        if minutes:
            total += int(minutes[0]) * 60
        if seconds:
            total += int(seconds[0])
        return total
    return pd.NA

# Clean each sheet
for sheet_name in xls.sheet_names:
    df_raw = pd.read_excel(file_path, sheet_name=sheet_name, header=None)
    
    # Detect correct header row
    header_row = None
    for i in range(10):
        row = df_raw.iloc[i]
        if "START" in row.values and "END" in row.values:
            header_row = i
            break

    if header_row is None:
        print(f"Skipping {sheet_name}: No header with START and END found.")
        continue

    # Load sheet with proper header
    df = pd.read_excel(file_path, sheet_name=sheet_name, header=header_row)
    
    # Drop rows with missing START or END
    df = df.dropna(subset=["START", "END"]).copy()

    # Convert distance and duration fields
    df["DISTANCE_M"] = df["DISTANCE"].apply(convert_distance)
    df["DURATION_SEC"] = df["DURATION"].apply(convert_duration)
    df["DURATION_IN_TRAFFIC_SEC"] = df["DURATION_IN_TRAFFIC"].apply(convert_duration)

    # Remove duplicates
    df = df.drop_duplicates()

    # Add time-based columns (if TIME column exists)
    if "


In [ ]:
import pandas as pd
import re

# Load the Excel file
file_path = "copy_whole_forward.xlsx"
xls = pd.ExcelFile(file_path)
cleaned_sheets = {}

# Conversion helpers
def convert_distance(dist):
    if isinstance(dist, str):
        if "km" in dist:
            num = float(dist.replace("km", "").strip())
            return int(num * 1000)
        elif "m" in dist:
            return int(dist.replace("m", "").strip())
    return pd.NA

def convert_duration(dur):
    if isinstance(dur, str):
        minutes = re.findall(r"(\d+)\s*min", dur)
        seconds = re.findall(r"(\d+)\s*sec", dur)
        total = 0
        if minutes:
            total += int(minutes[0]) * 60
        if seconds:
            total += int(seconds[0])
        return total
    return pd.NA

# Clean each sheet
for sheet_name in xls.sheet_names:
    df_raw = pd.read_excel(file_path, sheet_name=sheet_name, header=None)
    
    # Detect correct header row
    header_row = None
    for i in range(10):
        row = df_raw.iloc[i]
        if "START" in row.values and "END" in row.values:
            header_row = i
            break

    if header_row is None:
        print(f"Skipping {sheet_name}: No header with START and END found.")
        continue

    # Load sheet with proper header
    df = pd.read_excel(file_path, sheet_name=sheet_name, header=header_row)
    
    # Drop rows with missing START or END
    df = df.dropna(subset=["START", "END"]).copy()

    # Convert distance and duration fields
    df["DISTANCE_M"] = df["DISTANCE"].apply(convert_distance)
    df["DURATION_SEC"] = df["DURATION"].apply(convert_duration)
    df["DURATION_IN_TRAFFIC_SEC"] = df["DURATION_IN_TRAFFIC"].apply(convert_duration)

    # Remove duplicates
    df = df.drop_duplicates()

    # Add time-based columns (if TIME column exists)
    if "TIME" in df.columns:
        df["TIME"] = pd.to_datetime(df["TIME"], errors="coerce")
        df["DATE"] = df["TIME"].dt.date
        df["HOUR"] = df["TIME"].dt.hour
        df["WEEKDAY"] = df["TIME"].dt.day_name()

    cleaned_sheets[sheet_name] = df
    print(f"Cleaned {sheet_name}")

# Save cleaned data to a new Excel file
output_path = "cleaned_all_sheets_forward.xlsx"
with pd.ExcelWriter(output_path) as writer:
    for sheet_name, cleaned_df in cleaned_sheets.items():
        cleaned_df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"\n✅ Cleaning complete. Saved to: {output_path}")


In [ ]:
import pandas as pd
import re

# Load the Excel file
file_path = "backward_combined.xlsx"
xls = pd.ExcelFile(file_path)
cleaned_sheets = {}

# Conversion helpers
def convert_distance(dist):
    if isinstance(dist, str):
        if "km" in dist:
            num = float(dist.replace("km", "").strip())
            return int(num * 1000)
        elif "m" in dist:
            return int(dist.replace("m", "").strip())
    return pd.NA

def convert_duration(dur):
    if isinstance(dur, str):
        minutes = re.findall(r"(\d+)\s*min", dur)
        seconds = re.findall(r"(\d+)\s*sec", dur)
        total = 0
        if minutes:
            total += int(minutes[0]) * 60
        if seconds:
            total += int(seconds[0])
        return total
    return pd.NA

# Clean each sheet
for sheet_name in xls.sheet_names:
    df_raw = pd.read_excel(file_path, sheet_name=sheet_name, header=None)
    
    # Detect correct header row
    header_row = None
    for i in range(10):
        row = df_raw.iloc[i]
        if "START" in row.values and "END" in row.values:
            header_row = i
            break

    if header_row is None:
        print(f"Skipping {sheet_name}: No header with START and END found.")
        continue

    # Load sheet with proper header
    df = pd.read_excel(file_path, sheet_name=sheet_name, header=header_row)
    
    # Drop rows with missing START or END
    df = df.dropna(subset=["START", "END"]).copy()

    # Convert distance and duration fields
    df["DISTANCE_M"] = df["DISTANCE"].apply(convert_distance)
    df["DURATION_SEC"] = df["DURATION"].apply(convert_duration)
    df["DURATION_IN_TRAFFIC_SEC"] = df["DURATION_IN_TRAFFIC"].apply(convert_duration)

    # Remove duplicates
    df = df.drop_duplicates()

    # Add time-based columns (if TIME column exists)
    if "TIME" in df.columns:
        df["TIME"] = pd.to_datetime(df["TIME"], errors="coerce")
        df["DATE"] = df["TIME"].dt.date
        df["HOUR"] = df["TIME"].dt.hour
        df["WEEKDAY"] = df["TIME"].dt.day_name()

    cleaned_sheets[sheet_name] = df
    print(f"Cleaned {sheet_name}")

# Save cleaned data to a new Excel file
output_path = "cleaned_all_sheets_forward.xlsx"
with pd.ExcelWriter(output_path) as writer:
    for sheet_name, cleaned_df in cleaned_sheets.items():
        cleaned_df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"\n✅ Cleaning complete. Saved to: {output_path}")


In [ ]:
import pandas as pd
import re

# Load the Excel file
file_path = "copy_whole_forward.xlsx"
xls = pd.ExcelFile(file_path)
cleaned_sheets = {}

# Conversion helpers
def convert_distance(dist):
    if isinstance(dist, str):
        if "km" in dist:
            num = float(dist.replace("km", "").strip())
            return int(num * 1000)
        elif "m" in dist:
            return int(dist.replace("m", "").strip())
    return pd.NA

def convert_duration(dur):
    if isinstance(dur, str):
        minutes = re.findall(r"(\d+)\s*min", dur)
        seconds = re.findall(r"(\d+)\s*sec", dur)
        total = 0
        if minutes:
            total += int(minutes[0]) * 60
        if seconds:
            total += int(seconds[0])
        return total
    return pd.NA

# Clean each sheet
for sheet_name in xls.sheet_names:
    df_raw = pd.read_excel(file_path, sheet_name=sheet_name, header=None)
    
    # Detect correct header row
    header_row = None
    for i in range(10):
        row = df_raw.iloc[i]
        if "START" in row.values and "END" in row.values:
            header_row = i
            break

    if header_row is None:
        print(f"Skipping {sheet_name}: No header with START and END found.")
        continue

    # Load sheet with proper header
    df = pd.read_excel(file_path, sheet_name=sheet_name, header=header_row)
    
    # Drop rows with missing START or END
    df = df.dropna(subset=["START", "END"]).copy()

    # Convert distance and duration fields
    df["DISTANCE_M"] = df["DISTANCE"].apply(convert_distance)
    df["DURATION_SEC"] = df["DURATION"].apply(convert_duration)
    df["DURATION_IN_TRAFFIC_SEC"] = df["DURATION_IN_TRAFFIC"].apply(convert_duration)

    # Remove duplicates
    df = df.drop_duplicates()

    # Add time-based columns (if TIME column exists)
    if "TIME" in df.columns:
        df["TIME"] = pd.to_datetime(df["TIME"], errors="coerce")
        df["DATE"] = df["TIME"].dt.date
        df["HOUR"] = df["TIME"].dt.hour
        df["WEEKDAY"] = df["TIME"].dt.day_name()

    cleaned_sheets[sheet_name] = df
    print(f"Cleaned {sheet_name}")

# Save cleaned data to a new Excel file
output_path = "cleaned_all_sheets_forward.xlsx"
with pd.ExcelWriter(output_path) as writer:
    for sheet_name, cleaned_df in cleaned_sheets.items():
        cleaned_df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"\n✅ Cleaning complete. Saved to: {output_path}")


In [ ]:
import pandas as pd

# Load the Excel file
file_path = "cleaned_all_sheets_forward - Copy.xlsx"  # Update if your path is different
xls = pd.ExcelFile(file_path)

# Define the columns required for analysis
required_columns = ["START", "END", "DISTANCE_M", "DURATION_SEC", "DURATION_IN_TRAFFIC_SEC"]

# Check each sheet for readiness
for sheet_name in xls.sheet_names:
    df = xls.parse(sheet_name)
    missing = [col for col in required_columns if col not in df.columns]
    
    if missing:
        print(f"❌ {sheet_name} is missing: {missing}")
    else:
        print(f"✅ {sheet_name} is ready for analysis.")


In [ ]:
import pandas as pd

# Load the Excel file
file_path = "cleaned_all_sheets_backward - Copy.xlsx"  # Update if your path is different
xls = pd.ExcelFile(file_path)

# Define the columns required for analysis
required_columns = ["START", "END", "DISTANCE_M", "DURATION_SEC", "DURATION_IN_TRAFFIC_SEC"]

# Check each sheet for readiness
for sheet_name in xls.sheet_names:
    df = xls.parse(sheet_name)
    missing = [col for col in required_columns if col not in df.columns]
    
    if missing:
        print(f"❌ {sheet_name} is missing: {missing}")
    else:
        print(f"✅ {sheet_name} is ready for analysis.")


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set aesthetics for plots
sns.set(style="whitegrid")

# Load Excel file
file_path = "cleaned_all_sheets_forward - Copy.xlsx"
xls = pd.ExcelFile(file_path)

# Initialize empty DataFrame to combine all sheets
all_data = []

# Combine all data into one DataFrame
for sheet_name in xls.sheet_names:
    df = xls.parse(sheet_name)
    
    # Keep only necessary columns and remove rows with missing/zero durations
    df = df[["DISTANCE_M", "DURATION_SEC", "DURATION_IN_TRAFFIC_SEC"]].dropna()
    df = df[(df["DURATION_SEC"] > 0) & (df["DURATION_IN_TRAFFIC_SEC"] > 0)]
    df["Sheet"] = sheet_name  # Keep track of which sheet it came from
    all_data.append(df)

combined_df = pd.concat(all_data, ignore_index=True)

# Plotting Function
def plot_distribution(column_name, bins=50):
    plt.figure(figsize=(10, 6))
    sns.histplot(data=combined_df, x=column_name, bins=bins, kde=True, color='steelblue')
    plt.title(f"Distribution of {column_name}", fontsize=16)
    plt.xlabel(column_name)
    plt.ylabel("Frequency")
    plt.grid(True)
    plt.show()

# Plot distributions
plot_distribution("DISTANCE_M")
plot_distribution("DURATION_SEC")
plot_distribution("DURATION_IN_TRAFFIC_SEC")


In [ ]:
import pandas as pd

# Load cleaned Excel file
file_path = "cleaned_all_sheets_forward - Copy.xlsx"
xls = pd.ExcelFile(file_path)

all_data = []

for sheet_name in xls.sheet_names:
    df = xls.parse(sheet_name)

    # Check and drop rows with zero/negative values to avoid invalid calculations
    df = df.dropna(subset=["DISTANCE_M", "DURATION_SEC", "DURATION_IN_TRAFFIC_SEC"])
    df = df[(df["DISTANCE_M"] > 0) & (df["DURATION_SEC"] > 0) & (df["DURATION_IN_TRAFFIC_SEC"] > 0)]
    
    # Calculating traffic indices
    df["TDI"] = df["DURATION_IN_TRAFFIC_SEC"] / df["DURATION_SEC"]
    df["Delay_Sec"] = df["DURATION_IN_TRAFFIC_SEC"] - df["DURATION_SEC"]
    df["Normalized_Delay"] = df["Delay_Sec"] / df["DISTANCE_M"]
    df["Speed_kmph"] = (df["DISTANCE_M"] / 1000) / (df["DURATION_IN_TRAFFIC_SEC"] / 3600)

    df["Sheet"] = sheet_name
    all_data.append(df)

# Combine all sheets
combined_df = pd.concat(all_data, ignore_index=True)

# Save to Excel
combined_df.to_excel("indexed_traffic_data.xlsx", index=False)

print("✅ All indices calculated and saved to 'indexed_traffic_data.xlsx'")


In [ ]:
import pandas as pd

# Load cleaned Excel file
file_path = "cleaned_all_sheets_forward - Copy.xlsx"
xls = pd.ExcelFile(file_path)

# Create Excel writer for saving sheet-wise indexed data
with pd.ExcelWriter("indexed_sheetwise_forward.xlsx", engine="xlsxwriter") as writer:
    for sheet_name in xls.sheet_names:
        print(f"Processing sheet: {sheet_name}")
        df = xls.parse(sheet_name)

        # Drop invalid rows
        df = df.dropna(subset=["DISTANCE_M", "DURATION_SEC", "DURATION_IN_TRAFFIC_SEC"])
        df = df[(df["DISTANCE_M"] > 0) & (df["DURATION_SEC"] > 0) & (df["DURATION_IN_TRAFFIC_SEC"] > 0)]

        # Compute indices
        df["TDI"] = df["DURATION_IN_TRAFFIC_SEC"] / df["DURATION_SEC"]
        df["Delay_Sec"] = df["DURATION_IN_TRAFFIC_SEC"] - df["DURATION_SEC"]
        df["Normalized_Delay"] = df["Delay_Sec"] / df["DISTANCE_M"]
        df["Speed_kmph"] = (df["DISTANCE_M"] / 1000) / (df["DURATION_IN_TRAFFIC_SEC"] / 3600)

        # Save the modified DataFrame to the new Excel file
        df.to_excel(writer, sheet_name=sheet_name[:31], index=False)  # Excel sheet names must be <= 31 chars

print("✅ Done! Indexed data saved sheet-wise in 'indexed_sheetwise_forward.xlsx'")


In [ ]:
!pip install xlsxwriter


In [ ]:
import pandas as pd

# Load cleaned Excel file
file_path = "cleaned_all_sheets_forward - Copy.xlsx"
xls = pd.ExcelFile(file_path)

# Create Excel writer for saving sheet-wise indexed data
with pd.ExcelWriter("indexed_sheetwise_forward.xlsx", engine="xlsxwriter") as writer:
    for sheet_name in xls.sheet_names:
        print(f"Processing sheet: {sheet_name}")
        df = xls.parse(sheet_name)

        # Drop invalid rows
        df = df.dropna(subset=["DISTANCE_M", "DURATION_SEC", "DURATION_IN_TRAFFIC_SEC"])
        df = df[(df["DISTANCE_M"] > 0) & (df["DURATION_SEC"] > 0) & (df["DURATION_IN_TRAFFIC_SEC"] > 0)]

        # Compute indices
        df["TDI"] = df["DURATION_IN_TRAFFIC_SEC"] / df["DURATION_SEC"]
        df["Delay_Sec"] = df["DURATION_IN_TRAFFIC_SEC"] - df["DURATION_SEC"]
        df["Normalized_Delay"] = df["Delay_Sec"] / df["DISTANCE_M"]
        df["Speed_kmph"] = (df["DISTANCE_M"] / 1000) / (df["DURATION_IN_TRAFFIC_SEC"] / 3600)

        # Save the modified DataFrame to the new Excel file
        df.to_excel(writer, sheet_name=sheet_name[:31], index=False)  # Excel sheet names must be <= 31 chars

print("✅ Done! Indexed data saved sheet-wise in 'indexed_sheetwise_forward.xlsx'")


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the indexed Excel file
file_path = "indexed_sheetwise_forward.xlsx"
xls = pd.ExcelFile(file_path)

# Set Seaborn style
sns.set(style="whitegrid")

for sheet_name in xls.sheet_names:
    df = xls.parse(sheet_name)

    # Ensure required columns are there
    if "HOUR" in df.columns and "TDI" in df.columns:
        grouped = df.groupby("HOUR").agg({
            "TDI": "mean",
            "Delay_Sec": "mean",
            "Speed_kmph": "mean"
        }).reset_index()

        plt.figure(figsize=(12, 5))
        plt.suptitle(f"Hourly Traffic Impact – {sheet_name}", fontsize=16, fontweight="bold")

        # TDI
        plt.subplot(1, 3, 1)
        sns.barplot(x="HOUR", y="TDI", data=grouped, palette="Blues_d")
        plt.title("Avg TDI by Hour")
        plt.xticks(rotation=45)

        # Delay
        plt.subplot(1, 3, 2)
        sns.barplot(x="HOUR", y="Delay_Sec", data=grouped, palette="Oranges_d")
        plt.title("Avg Delay (Sec) by Hour")
        plt.xticks(rotation=45)

        # Speed
        plt.subplot(1, 3, 3)
        sns.barplot(x="HOUR", y="Speed_kmph", data=grouped, palette="Greens_d")
        plt.title("Avg Speed (km/h) by Hour")
        plt.xticks(rotation=45)

        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the Excel file
file_path = "indexed_sheetwise_forward.xlsx"
xls = pd.ExcelFile(file_path)

sns.set(style="whitegrid")  # Seaborn style for clean plots

for sheet_name in xls.sheet_names:
    df = xls.parse(sheet_name)

    if "HOUR" in df.columns and "TDI" in df.columns:
        grouped = df.groupby("HOUR").agg({
            "TDI": "mean",
            "Delay_Sec": "mean",
            "Speed_kmph": "mean"
        }).reset_index()

        plt.figure(figsize=(12, 5))
        plt.suptitle(f"Hourly Traffic Impact – {sheet_name}", fontsize=16, fontweight="bold")

        # TDI
        plt.subplot(1, 3, 1)
        sns.barplot(x="HOUR", y="TDI", data=grouped, palette="Blues_d")
        plt.title("Avg TDI by Hour")

        # Delay
        plt.subplot(1, 3, 2)
        sns.barplot(x="HOUR", y="Delay_Sec", data=grouped, palette="Oranges_d")
        plt.title("Avg Delay (Sec) by Hour")

        # Speed
        plt.subplot(1, 3, 3)
        sns.barplot(x="HOUR", y="Speed_kmph", data=grouped, palette="Greens_d")
        plt.title("Avg Speed (km/h) by Hour")

        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        plt.show()  # ← this ensures the plot appears


In [ ]:
# Add this if using Jupyter
%matplotlib inline


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the Excel file
file_path = "indexed_sheetwise_forward.xlsx"
xls = pd.ExcelFile(file_path)

sns.set(style="whitegrid")  # Seaborn style for clean plots

for sheet_name in xls.sheet_names:
    df = xls.parse(sheet_name)

    if "HOUR" in df.columns and "TDI" in df.columns:
        grouped = df.groupby("HOUR").agg({
            "TDI": "mean",
            "Delay_Sec": "mean",
            "Speed_kmph": "mean"
        }).reset_index()

        plt.figure(figsize=(12, 5))
        plt.suptitle(f"Hourly Traffic Impact – {sheet_name}", fontsize=16, fontweight="bold")

        # TDI
        plt.subplot(1, 3, 1)
        sns.barplot(x="HOUR", y="TDI", data=grouped, palette="Blues_d")
        plt.title("Avg TDI by Hour")

        # Delay
        plt.subplot(1, 3, 2)
        sns.barplot(x="HOUR", y="Delay_Sec", data=grouped, palette="Oranges_d")
        plt.title("Avg Delay (Sec) by Hour")

        # Speed
        plt.subplot(1, 3, 3)
        sns.barplot(x="HOUR", y="Speed_kmph", data=grouped, palette="Greens_d")
        plt.title("Avg Speed (km/h) by Hour")

        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        plt.show()  # ← this ensures the plot appears


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Load Excel file
file_path = "indexed_sheetwise_forward.xlsx"
xls = pd.ExcelFile(file_path)

# Create output directory to save plots
output_dir = "hourly_traffic_plots"
os.makedirs(output_dir, exist_ok=True)

sns.set(style="whitegrid")

for sheet_name in xls.sheet_names:
    df = xls.parse(sheet_name)

    if "HOUR" in df.columns and "TDI" in df.columns:
        grouped = df.groupby("HOUR").agg({
            "TDI": "mean",
            "Delay_Sec": "mean",
            "Speed_kmph": "mean"
        }).reset_index()

        plt.figure(figsize=(12, 5))
        plt.suptitle(f"Hourly Traffic Impact – {sheet_name}", fontsize=16, fontweight="bold")

        # TDI
        plt.subplot(1, 3, 1)
        sns.barplot(x="HOUR", y="TDI", data=grouped, palette="Blues_d")
        plt.title("Avg TDI by Hour")

        # Delay
        plt.subplot(1, 3, 2)
        sns.barplot(x="HOUR", y="Delay_Sec", data=grouped, palette="Oranges_d")
        plt.title("Avg Delay (Sec) by Hour")

        # Speed
        plt.subplot(1, 3, 3)
        sns.barplot(x="HOUR", y="Speed_kmph", data=grouped, palette="Greens_d")
        plt.title("Avg Speed (km/h) by Hour")

        plt.tight_layout(rect=[0, 0.03, 1, 0.95])

        # Save the plot
        file_name = f"{sheet_name.replace('/', '_')}.png"
        plt.savefig(os.path.join(output_dir, file_name))
        plt.close()

print(f"✅ All plots saved in the '{output_dir}' folder.")


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# Path to your indexed Excel file
file_path = "indexed_sheetwise_forward.xlsx"

# Create folder to save plots
output_folder = "hourly_traffic_plots"
os.makedirs(output_folder, exist_ok=True)

# Load Excel file
xls = pd.ExcelFile(file_path)

# Loop through each sheet
for sheet_name in xls.sheet_names:
    df = xls.parse(sheet_name)

    # Skip if HOUR or DURATION_IN_TRAFFIC_SEC missing
    if 'HOUR' not in df.columns or 'DURATION_IN_TRAFFIC_SEC' not in df.columns:
        print(f"Skipping {sheet_name} (missing required columns)")
        continue

    # Group by hour and calculate average duration in traffic
    hourly_avg = df.groupby("HOUR")["DURATION_IN_TRAFFIC_SEC"].mean()

    # Plot
    plt.figure(figsize=(10, 5))
    hourly_avg.plot(kind='bar', color='skyblue')
    plt.title(f"Avg. Traffic Duration by Hour\n{sheet_name}")
    plt.xlabel("Hour of Day")
    plt.ylabel("Avg. Duration in Traffic (seconds)")
    plt.grid(axis='y')
    plt.tight_layout()

    # Save plot
    plot_path = os.path.join(output_folder, f"{sheet_name}_hourly_traffic.png")
    plt.savefig(plot_path)
    plt.close()

    print(f"Saved: {plot_path}")


In [ ]:
import pandas as pd

file_path = "indexed_sheetwise_forward.xlsx"
xls = pd.ExcelFile(file_path)

for sheet in xls.sheet_names:
    df = xls.parse(sheet, nrows=1)  # just read header
    print(f"{sheet}: {list(df.columns)}")


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

file_path = "indexed_sheetwise_forward.xlsx"
output_folder = "hourly_traffic_plots"
os.makedirs(output_folder, exist_ok=True)

# Load all sheet names
xls = pd.ExcelFile(file_path)

for sheet_name in xls.sheet_names:
    df = xls.parse(sheet_name)
    
    if "DATE&TIME" in df.columns and "DURATION_IN_TRAFFIC_SEC" in df.columns:
        # Convert date-time column
        df["DATE&TIME"] = pd.to_datetime(df["DATE&TIME"], errors="coerce")
        df["HOUR"] = df["DATE&TIME"].dt.hour

        # Group by hour
        hourly_avg = df.groupby("HOUR")["DURATION_IN_TRAFFIC_SEC"].mean()

        # Plot
        plt.figure(figsize=(10, 5))
        hourly_avg.plot(kind='bar', color='steelblue')
        plt.title(f"Average Traffic Duration by Hour\n{sheet_name}")
        plt.xlabel("Hour of Day")
        plt.ylabel("Avg Duration in Traffic (sec)")
        plt.xticks(rotation=0)
        plt.tight_layout()

        # Save plot
        plot_path = os.path.join(output_folder, f"{sheet_name}_hourly_traffic.png")
        plt.savefig(plot_path)
        plt.close()
        print(f"Saved plot for {sheet_name}")
        
    else:
        print(f"Skipping {sheet_name} (missing DATE&TIME or DURATION_IN_TRAFFIC_SEC)")


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

file_path = "indexed_sheetwise_forward.xlsx"
output_folder = "daily_traffic_plots"
os.makedirs(output_folder, exist_ok=True)

# Load all sheet names
xls = pd.ExcelFile(file_path)

for sheet_name in xls.sheet_names:
    df = xls.parse(sheet_name)
    
    if "DATE&TIME" in df.columns and "DURATION_IN_TRAFFIC_SEC" in df.columns:
        # Convert date-time column
        df["DATE&TIME"] = pd.to_datetime(df["DATE&TIME"], errors="coerce")
        df["DATE"] = df["DATE&TIME"].dt.date  # Extract date (day) from datetime

        # Group by date
        daily_avg = df.groupby("DATE")["DURATION_IN_TRAFFIC_SEC"].mean()

        # Plot
        plt.figure(figsize=(12, 6))
        daily_avg.plot(kind='bar', color='steelblue')
        plt.title(f"Average Traffic Duration by Day\n{sheet_name}")
        plt.xlabel("Date")
        plt.ylabel("Avg Duration in Traffic (sec)")
        plt.xticks(rotation=45)
        plt.tight_layout()

        # Save plot
        plot_path = os.path.join(output_folder, f"{sheet_name}_daily_traffic.png")
        plt.savefig(plot_path)
        plt.close()
        print(f"Saved plot for {sheet_name}")
        
    else:
        print(f"Skipping {sheet_name} (missing DATE&TIME or DURATION_IN_TRAFFIC_SEC)")


In [ ]:
import matplotlib.dates as mdates

# Plot
plt.figure(figsize=(12, 6))
plt.plot(daily_avg.index, daily_avg.values, marker='o', linestyle='-', color='steelblue')
plt.title(f"Average Traffic Duration by Day\n{sheet_name}")
plt.xlabel("Date")
plt.ylabel("Avg Duration in Traffic (sec)")

# Format x-axis
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())
plt.gcf().autofmt_xdate()  # Auto-rotate dates nicely

plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()

# Save plot
plot_path = os.path.join(output_folder, f"{sheet_name}_daily_traffic.png")
plt.savefig(plot_path)
plt.close()
print(f"Saved plot for {sheet_name}")


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import os

file_path = "indexed_sheetwise_forward.xlsx"
output_folder = "daily_traffic_plots"
os.makedirs(output_folder, exist_ok=True)

# Load all sheet names
xls = pd.ExcelFile(file_path)

for sheet_name in xls.sheet_names:
    df = xls.parse(sheet_name)
    
    if "DATE&TIME" in df.columns and "DURATION_IN_TRAFFIC_SEC" in df.columns:
        # Convert date-time column
        df["DATE&TIME"] = pd.to_datetime(df["DATE&TIME"], errors="coerce")
        df["DATE"] = df["DATE&TIME"].dt.date  # Extract date

        # Group by date
        daily_avg = df.groupby("DATE")["DURATION_IN_TRAFFIC_SEC"].mean()

        # Optional: Smooth with 7-day rolling average
        # daily_avg = daily_avg.rolling(window=7, min_periods=1).mean()

        # Plot
        plt.figure(figsize=(12, 6))
        plt.plot(daily_avg.index, daily_avg.values, marker='o', linestyle='-', color='steelblue')
        plt.title(f"Average Traffic Duration by Day\n{sheet_name}")
        plt.xlabel("Date")
        plt.ylabel("Avg Duration in Traffic (sec)")

        # Format x-axis
        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
        plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())
        plt.gcf().autofmt_xdate()
        plt.grid(True, linestyle='--', alpha=0.6)
        plt.tight_layout()

        # Sanitize sheet name for saving
        safe_sheet_name = "".join(c if c.isalnum() or c in "._-" else "_" for c in sheet_name)
        plot_path = os.path.join(output_folder, f"{safe_sheet_name}_daily_traffic.png")
        plt.savefig(plot_path)
        plt.close()
        print(f"Saved plot for {sheet_name}")

    else:
        print(f"Skipping {sheet_name} (missing DATE&TIME or DURATION_IN_TRAFFIC_SEC)")


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import os

file_path = "indexed_sheetwise_forward.xlsx"
output_folder = "combined_plots"
os.makedirs(output_folder, exist_ok=True)

xls = pd.ExcelFile(file_path)

# Store daily average delay per sheet
daily_delays = {}

for sheet_name in xls.sheet_names:
    df = xls.parse(sheet_name)

    if "DATE&TIME" in df.columns and "DURATION_IN_TRAFFIC_SEC" in df.columns:
        df["DATE&TIME"] = pd.to_datetime(df["DATE&TIME"], errors="coerce")
        df["DATE"] = df["DATE&TIME"].dt.date
        daily_avg = df.groupby("DATE")["DURATION_IN_TRAFFIC_SEC"].mean()
        daily_delays[sheet_name] = daily_avg
    else:
        print(f"Skipping {sheet_name} (missing required columns)")

# Combine all into one DataFrame
combined_df = pd.DataFrame(daily_delays)
combined_df.dropna(inplace=True)  # Ensure all stretches have data

# Calculate the average delay across all stretches for each day
combined_df["AVG_ALL_STRETCHES"] = combined_df.mean(axis=1)

# Find day with max and min
max_day = combined_df["AVG_ALL_STRETCHES"].idxmax()
min_day = combined_df["AVG_ALL_STRETCHES"].idxmin()

# Plot
plt.figure(figsize=(14, 6))
plt.plot(combined_df.index, combined_df["AVG_ALL_STRETCHES"], marker='o', linestyle='-', color='darkorange')
plt.title("Average Delay Across All Stretches by Day", fontsize=14)
plt.xlabel("Date")
plt.ylabel("Avg Delay (sec)")
plt.grid(True, linestyle='--', alpha=0.6)

# Mark max and min
plt.axvline(max_day, color='red', linestyle='--', label=f'Max: {max_day} ({combined_df["AVG_ALL_STRETCHES"].max():.0f} sec)')
plt.axvline(min_day, color='green', linestyle='--', label=f'Min: {min_day} ({combined_df["AVG_ALL_STRETCHES"].min():.0f} sec)')

# Format x-axis
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
plt.gcf().autofmt_xdate()

plt.legend()
plt.tight_layout()

# Save plot
plot_path = os.path.join(output_folder, "combined_daily_avg_delay.png")
plt.savefig(plot_path)
plt.close()
print(f"✅ Plot saved at: {plot_path}")


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# Load Excel file
file_path = "indexed_sheetwise_forward.xlsx"  # Change path if needed
xls = pd.ExcelFile(file_path)

# Define sheet names for Flyover 1
flyover_sheets = ["Forward 1_Sheet4", "Forward 1_Sheet5"]
underpass_sheets = ["Forward 1_Sheet6", "Forward 1_Sheet7", "Forward 2_Sheet1"]

# Create output directory
output_folder = "flyover_1_analysis"
os.makedirs(output_folder, exist_ok=True)

# Helper function to combine data from multiple sheets
def get_combined_df(sheet_list):
    dfs = []
    for sheet in sheet_list:
        if sheet in xls.sheet_names:
            df = xls.parse(sheet)
            if "DATE&TIME" in df.columns and "DURATION_IN_TRAFFIC_SEC" in df.columns:
                df["DATE&TIME"] = pd.to_datetime(df["DATE&TIME"], errors="coerce")
                df = df.dropna(subset=["DATE&TIME"])
                df["HOUR"] = df["DATE&TIME"].dt.hour
                df["DAY"] = df["DATE&TIME"].dt.day_name()
                dfs.append(df)
    return pd.concat(dfs, ignore_index=True)

# Read and combine data
flyover_df = get_combined_df(flyover_sheets)
underpass_df = get_combined_df(underpass_sheets)

# Hourly average comparison
flyover_hourly = flyover_df.groupby("HOUR")["DURATION_IN_TRAFFIC_SEC"].mean()
underpass_hourly = underpass_df.groupby("HOUR")["DURATION_IN_TRAFFIC_SEC"].mean()

plt.figure(figsize=(12, 6))
plt.plot(flyover_hourly.index, flyover_hourly.values, marker='o', label='Flyover (Above)')
plt.plot(underpass_hourly.index, underpass_hourly.values, marker='o', label='Underpass (Below)')
plt.title("Flyover vs Underpass - Hourly Avg Traffic Delay (Flyover 1)")
plt.xlabel("Hour of Day")
plt.ylabel("Avg Delay (sec)")
plt.legend()
plt.grid(True)
plt.xticks(range(0, 24))
plt.tight_layout()
plt.savefig(os.path.join(output_folder, "hourly_comparison.png"))
plt.show()

# Day-wise average comparison
flyover_daily = flyover_df.groupby("DAY")["DURATION_IN_TRAFFIC_SEC"].mean()
underpass_daily = underpass_df.groupby("DAY")["DURATION_IN_TRAFFIC_SEC"].mean()

# Ensure proper day order
day_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
flyover_daily = flyover_daily.reindex(day_order)
underpass_daily = underpass_daily.reindex(day_order)

plt.figure(figsize=(12, 6))
plt.plot(flyover_daily.index, flyover_daily.values, marker='o', label='Flyover (Above)')
plt.plot(underpass_daily.index, underpass_daily.values, marker='o', label='Underpass (Below)')
plt.title("Flyover vs Underpass - Day-wise Avg Traffic Delay (Flyover 1)")
plt.xlabel("Day of Week")
plt.ylabel("Avg Delay (sec)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(os.path.join(output_folder, "daywise_comparison.png"))
plt.show()


In [ ]:
# Convert datetime to just date (for daily average)
flyover_df["DATE"] = flyover_df["DATE&TIME"].dt.date
underpass_df["DATE"] = underpass_df["DATE&TIME"].dt.date

# Daily average traffic delay
flyover_datewise = flyover_df.groupby("DATE")["DURATION_IN_TRAFFIC_SEC"].mean()
underpass_datewise = underpass_df.groupby("DATE")["DURATION_IN_TRAFFIC_SEC"].mean()

# Plotting
plt.figure(figsize=(14, 6))
plt.plot(flyover_datewise.index, flyover_datewise.values, marker='o', linestyle='-', label='Flyover (Above)')
plt.plot(underpass_datewise.index, underpass_datewise.values, marker='o', linestyle='-', label='Underpass (Below)')
plt.title("Flyover vs Underpass - Daily Avg Traffic Delay (Flyover 1)")
plt.xlabel("Date")
plt.ylabel("Avg Delay (sec)")
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(os.path.join(output_folder, "datewise_comparison.png"))
plt.show()



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# Flyover 2 sheet mapping
above_sheets = ["Forward 2_Sheet5", "Forward 2_Sheet6"]
below_sheets = ["Forward 2_Sheet7", "Forward 3_Sheet1", "Forward 3_Sheet2"]

# Path to the Excel file
file_path = "indexed_sheetwise_forward.xlsx"

# Create output folder
output_folder = "flyover_2_analysis"
os.makedirs(output_folder, exist_ok=True)

# Load the Excel file
xls = pd.ExcelFile(file_path)

def extract_combined_data(sheet_names):
    """Extracts and combines data from specified sheets into a single DataFrame."""
    combined_df = pd.DataFrame()
    for sheet in sheet_names:
        if sheet not in xls.sheet_names:
            print(f"Warning: Sheet {sheet} not found in the file.")
            continue
        df = xls.parse(sheet)
        if "DATE&TIME" not in df.columns or "DURATION_IN_TRAFFIC_SEC" not in df.columns:
            print(f"Sheet {sheet} is missing required columns.")
            continue
        df["DATE&TIME"] = pd.to_datetime(df["DATE&TIME"], errors="coerce")
        df.dropna(subset=["DATE&TIME", "DURATION_IN_TRAFFIC_SEC"], inplace=True)
        df["DATE"] = df["DATE&TIME"].dt.date
        df["HOUR"] = df["DATE&TIME"].dt.hour
        df["WEEKDAY"] = df["DATE&TIME"].dt.day_name()
        combined_df = pd.concat([combined_df, df], ignore_index=True)
    return combined_df

# Extract data for above and below flyover paths
above_df = extract_combined_data(above_sheets)
below_df = extract_combined_data(below_sheets)

# Plot 1: Weekday-wise Average Delay
above_weekday = above_df.groupby("WEEKDAY")["DURATION_IN_TRAFFIC_SEC"].mean()
below_weekday = below_df.groupby("WEEKDAY")["DURATION_IN_TRAFFIC_SEC"].mean()
ordered_days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
above_weekday = above_weekday.reindex(ordered_days)
below_weekday = below_weekday.reindex(ordered_days)

plt.figure(figsize=(10, 6))
plt.plot(above_weekday.index, above_weekday.values, label="Above Flyover", marker='o')
plt.plot(below_weekday.index, below_weekday.values, label="Below Flyover", marker='o')
plt.title("Weekday-wise Avg Traffic Delay - Flyover 2")
plt.xlabel("Weekday")
plt.ylabel("Avg Delay (sec)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(os.path.join(output_folder, "weekday_comparison.png"))
plt.close()

# Plot 2: Hour-wise Average Delay
above_hour = above_df.groupby("HOUR")["DURATION_IN_TRAFFIC_SEC"].mean()
below_hour = below_df.groupby("HOUR")["DURATION_IN_TRAFFIC_SEC"].mean()

plt.figure(figsize=(10, 6))
plt.plot(above_hour.index, above_hour.values, label="Above Flyover", marker='o')
plt.plot(below_hour.index, below_hour.values, label="Below Flyover", marker='o')
plt.title("Hour-wise Avg Traffic Delay - Flyover 2")
plt.xlabel("Hour of Day")
plt.ylabel("Avg Delay (sec)")
plt.xticks(range(0, 24))
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(os.path.join(output_folder, "hourly_comparison.png"))
plt.close()

# Plot 3: Day-wise Delay Trends (Actual Dates)
above_daily = above_df.groupby("DATE")["DURATION_IN_TRAFFIC_SEC"].mean()
below_daily = below_df.groupby("DATE")["DURATION_IN_TRAFFIC_SEC"].mean()

plt.figure(figsize=(12, 6))
plt.plot(above_daily.index, above_daily.values, label="Above Flyover", marker='.')
plt.plot(below_daily.index, below_daily.values, label="Below Flyover", marker='.')
plt.title("Daily Avg Delay Trend - Flyover 2")
plt.xlabel("Date")
plt.ylabel("Avg Delay (sec)")
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.grid(True)
plt.savefig(os.path.join(output_folder, "daily_trend_comparison.png"))
plt.close()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# Define file and output folder
file_path = "indexed_sheetwise_forward.xlsx"
output_folder = "flyover_3_analysis"
os.makedirs(output_folder, exist_ok=True)

# Sheets going above and below the flyover
above_sheets = ["Forward 3_Sheet4", "Forward 3_Sheet5", "Forward 3_Sheet6"]
below_sheets = ["Forward 3_Sheet7", "Forward 4_Sheet1", "Forward 4_Sheet2", "Forward 4_Sheet3"]

# Load Excel
xls = pd.ExcelFile(file_path)

def get_daily_avg(sheet_names):
    combined_df = pd.DataFrame()
    for sheet in sheet_names:
        df = xls.parse(sheet)
        if "DATE&TIME" in df.columns and "DURATION_IN_TRAFFIC_SEC" in df.columns:
            df["DATE&TIME"] = pd.to_datetime(df["DATE&TIME"], errors="coerce")
            df.dropna(subset=["DATE&TIME"], inplace=True)
            df["DATE"] = df["DATE&TIME"].dt.date
            daily_avg = df.groupby("DATE")["DURATION_IN_TRAFFIC_SEC"].mean().reset_index()
            daily_avg.rename(columns={"DURATION_IN_TRAFFIC_SEC": sheet}, inplace=True)
            if combined_df.empty:
                combined_df = daily_avg
            else:
                combined_df = pd.merge(combined_df, daily_avg, on="DATE", how="outer")
    combined_df.set_index("DATE", inplace=True)
    return combined_df

# Get daily averages
above_df = get_daily_avg(above_sheets)
below_df = get_daily_avg(below_sheets)

# Calculate mean across stretches
above_mean = above_df.mean(axis=1)
below_mean = below_df.mean(axis=1)

# Plot hourly comparison
plt.figure(figsize=(12, 6))
above_mean.plot(label="Above Flyover", linewidth=2)
below_mean.plot(label="Below Flyover", linewidth=2)
plt.title("Flyover 3 - Daily Avg Delay (All Days)")
plt.xlabel("Date")
plt.ylabel("Avg Duration in Traffic (sec)")
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(os.path.join(output_folder, "flyover3_daily_comparison.png"))
plt.close()

# Day-wise comparison (e.g. Monday, Tuesday, ...)
above_df_daywise = above_df.copy()
below_df_daywise = below_df.copy()
above_df_daywise["Day"] = above_df_daywise.index.to_series().apply(lambda x: x.strftime("%A"))
below_df_daywise["Day"] = below_df_daywise.index.to_series().apply(lambda x: x.strftime("%A"))
above_daywise = above_df_daywise.groupby("Day").mean().mean(axis=1)
below_daywise = below_df_daywise.groupby("Day").mean().mean(axis=1)

# Plot day-wise comparison
plt.figure(figsize=(10, 5))
above_daywise = above_daywise.loc[["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]]
below_daywise = below_daywise.loc[["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]]
plt.plot(above_daywise.index, above_daywise.values, label="Above Flyover", marker='o')
plt.plot(below_daywise.index, below_daywise.values, label="Below Flyover", marker='o')
plt.title("Flyover 3 - Avg Delay by Day of Week")
plt.ylabel("Avg Duration in Traffic (sec)")
plt.xlabel("Day of Week")
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(output_folder, "flyover3_daywise_comparison.png"))
plt.close()

# Plot full daily line chart
plt.figure(figsize=(14, 6))
for sheet in above_sheets:
    if sheet in above_df.columns:
        plt.plot(above_df.index, above_df[sheet], label=f"Above - {sheet}")
for sheet in below_sheets:
    if sheet in below_df.columns:
        plt.plot(below_df.index, below_df[sheet], label=f"Below - {sheet}")
plt.title("Flyover 3 - Daily Delay Trends (All Sheets)")
plt.xlabel("Date")
plt.ylabel("Avg Duration in Traffic (sec)")
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(output_folder, "flyover3_all_sheet_trends.png"))
plt.close()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# File and sheet information
file_path = "indexed_sheetwise_forward.xlsx"
output_folder = "flyover_3_analysis"
os.makedirs(output_folder, exist_ok=True)

# Define sheets for above and below
above_sheets = ["Forward 3_Sheet4", "Forward 3_Sheet5", "Forward 3_Sheet6"]
below_sheets = ["Forward 3_Sheet7", "Forward 4_Sheet1", "Forward 4_Sheet2", "Forward 4_Sheet3"]

# Load Excel file
xls = pd.ExcelFile(file_path)

def load_and_process_sheets(sheet_list):
    dfs = []
    for sheet in sheet_list:
        df = xls.parse(sheet)
        if "DATE&TIME" in df.columns and "DURATION_IN_TRAFFIC_SEC" in df.columns:
            df["DATE&TIME"] = pd.to_datetime(df["DATE&TIME"], errors="coerce")
            df.dropna(subset=["DATE&TIME"], inplace=True)
            df["HOUR"] = df["DATE&TIME"].dt.hour
            df["DATE"] = df["DATE&TIME"].dt.date
            dfs.append(df)
    return pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()

# Process both above and below
above_df = load_and_process_sheets(above_sheets)
below_df = load_and_process_sheets(below_sheets)

# Plot 1: Hour-wise comparison
above_hourly = above_df.groupby("HOUR")["DURATION_IN_TRAFFIC_SEC"].mean()
below_hourly = below_df.groupby("HOUR")["DURATION_IN_TRAFFIC_SEC"].mean()

plt.figure(figsize=(12, 6))
plt.plot(above_hourly.index, above_hourly.values, label="Above Flyover", marker='o')
plt.plot(below_hourly.index, below_hourly.values, label="Below Flyover", marker='o')
plt.title("Flyover 3 - Hourly Average Delay Comparison")
plt.xlabel("Hour of Day")
plt.ylabel("Average Delay (sec)")
plt.legend()
plt.grid(True)
plt.xticks(range(24))
plt.tight_layout()
plt.savefig(os.path.join(output_folder, "flyover_3_hourly_comparison.png"))
plt.close()

# Plot 2: Day of Week Comparison
above_df["DAY"] = pd.to_datetime(above_df["DATE"])
below_df["DAY"] = pd.to_datetime(below_df["DATE"])
above_df["WEEKDAY"] = above_df["DAY"].dt.day_name()
below_df["WEEKDAY"] = below_df["DAY"].dt.day_name()

above_weekday = above_df.groupby("WEEKDAY")["DURATION_IN_TRAFFIC_SEC"].mean()
below_weekday = below_df.groupby("WEEKDAY")["DURATION_IN_TRAFFIC_SEC"].mean()

# Ensure consistent weekday order
weekday_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
above_weekday = above_weekday.reindex(weekday_order)
below_weekday = below_weekday.reindex(weekday_order)

plt.figure(figsize=(12, 6))
plt.plot(above_weekday.index, above_weekday.values, label="Above Flyover", marker='o')
plt.plot(below_weekday.index, below_weekday.values, label="Below Flyover", marker='o')
plt.title("Flyover 3 - Day of Week Average Delay Comparison")
plt.xlabel("Day of Week")
plt.ylabel("Average Delay (sec)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(os.path.join(output_folder, "flyover_3_weekday_comparison.png"))
plt.close()

# Plot 3: Daily Trend Line Plot
above_daily = above_df.groupby("DATE")["DURATION_IN_TRAFFIC_SEC"].mean()
below_daily = below_df.groupby("DATE")["DURATION_IN_TRAFFIC_SEC"].mean()

plt.figure(figsize=(14, 6))
plt.plot(above_daily.index, above_daily.values, label="Above Flyover", marker='o')
plt.plot(below_daily.index, below_daily.values, label="Below Flyover", marker='o')
plt.title("Flyover 3 - Daily Average Delay Trend")
plt.xlabel("Date")
plt.ylabel("Average Delay (sec)")
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(os.path.join(output_folder, "flyover_3_daily_trend.png"))
plt.close()

In [ ]:
# Flyover 4 Analysis Script

import pandas as pd
import matplotlib.pyplot as plt
import os

# File and sheet details
file_path = "indexed_sheetwise_forward.xlsx"
sheets_above = ["Forward 4_Sheet6", "Forward 4_Sheet7"]
sheets_below = ["Forward 5_Sheet1", "Forward 5_Sheet2", "Forward 5_Sheet3"]
output_folder = "flyover_4_analysis"
os.makedirs(output_folder, exist_ok=True)

# Load Excel file
xls = pd.ExcelFile(file_path)

# Helper function to load and concatenate sheets
def load_and_prepare_sheets(sheet_names):
    dfs = []
    for name in sheet_names:
        df = xls.parse(name)
        if "DATE&TIME" in df.columns and "DURATION_IN_TRAFFIC_SEC" in df.columns:
            df["DATE&TIME"] = pd.to_datetime(df["DATE&TIME"], errors='coerce')
            df = df.dropna(subset=["DATE&TIME", "DURATION_IN_TRAFFIC_SEC"])
            df["DATE"] = df["DATE&TIME"].dt.date
            df["HOUR"] = df["DATE&TIME"].dt.hour
            dfs.append(df)
    return pd.concat(dfs, ignore_index=True)

# Load data
above_df = load_and_prepare_sheets(sheets_above)
below_df = load_and_prepare_sheets(sheets_below)

# 1. Hour-wise average comparison
hourly_above = above_df.groupby("HOUR")["DURATION_IN_TRAFFIC_SEC"].mean()
hourly_below = below_df.groupby("HOUR")["DURATION_IN_TRAFFIC_SEC"].mean()

plt.figure(figsize=(12, 6))
plt.plot(hourly_above.index, hourly_above.values, label='Above (Flyover)', marker='o')
plt.plot(hourly_below.index, hourly_below.values, label='Below (Underpass)', marker='o')
plt.xlabel("Hour of Day")
plt.ylabel("Avg Duration in Traffic (sec)")
plt.title("Flyover 4: Hour-wise Avg Traffic Delay")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(os.path.join(output_folder, "flyover_4_hourly_comparison.png"))
plt.close()

# 2. Day-wise average comparison
daywise_above = above_df.groupby("DATE")["DURATION_IN_TRAFFIC_SEC"].mean()
daywise_below = below_df.groupby("DATE")["DURATION_IN_TRAFFIC_SEC"].mean()

plt.figure(figsize=(12, 6))
plt.plot(daywise_above.index, daywise_above.values, label='Above (Flyover)', marker='o')
plt.plot(daywise_below.index, daywise_below.values, label='Below (Underpass)', marker='o')
plt.xlabel("Date")
plt.ylabel("Avg Duration in Traffic (sec)")
plt.title("Flyover 4: Day-wise Avg Traffic Delay")
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(os.path.join(output_folder, "flyover_4_daywise_comparison.png"))
plt.close()

# 3. Daily line plot for each (not grouped by weekday)
plt.figure(figsize=(12, 6))
plt.plot(daywise_above.index, daywise_above.values, label='Above (Flyover)', marker='o')
plt.plot(daywise_below.index, daywise_below.values, label='Below (Underpass)', marker='o')
plt.xlabel("Date")
plt.ylabel("Avg Duration in Traffic (sec)")
plt.title("Flyover 4: Daily Avg Traffic Duration Line Plot")
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(os.path.join(output_folder, "flyover_4_daily_line_plot.png"))
plt.close()

print("Saved all Flyover 4 comparison plots in:", output_folder)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# File and folder setup
file_path = "indexed_sheetwise_forward.xlsx"
output_folder = "flyover_4_analysis"
os.makedirs(output_folder, exist_ok=True)

# Sheets for flyover 4
above_sheets = ["Forward 4_Sheet6", "Forward 4_Sheet7"]
below_sheets = ["Forward 5_Sheet1", "Forward 5_Sheet2", "Forward 5_Sheet3"]

# Load Excel file
xls = pd.ExcelFile(file_path)

# Helper function to aggregate all required data from sheets
def get_combined_df(sheet_names):
    df_list = []
    for sheet in sheet_names:
        df = xls.parse(sheet)
        if "DATE&TIME" in df.columns and "DURATION_IN_TRAFFIC_SEC" in df.columns:
            df["DATE&TIME"] = pd.to_datetime(df["DATE&TIME"], errors='coerce')
            df = df.dropna(subset=["DATE&TIME", "DURATION_IN_TRAFFIC_SEC"])
            df_list.append(df[["DATE&TIME", "DURATION_IN_TRAFFIC_SEC"]])
        else:
            print(f"Skipping {sheet} due to missing columns.")
    return pd.concat(df_list)

# Get data
above_df = get_combined_df(above_sheets)
below_df = get_combined_df(below_sheets)

# HOURLY AVERAGE PLOT
def plot_hourly(df, label):
    df["HOUR"] = df["DATE&TIME"].dt.hour
    return df.groupby("HOUR")["DURATION_IN_TRAFFIC_SEC"].mean()

above_hourly = plot_hourly(above_df, "Above")
below_hourly = plot_hourly(below_df, "Below")

plt.figure(figsize=(12, 6))
plt.plot(above_hourly.index, above_hourly.values, label="Above Flyover", marker='o')
plt.plot(below_hourly.index, below_hourly.values, label="Below Flyover", marker='o')
plt.title("Flyover 4 - Hourly Avg Traffic Delay (sec)")
plt.xlabel("Hour of Day")
plt.ylabel("Avg Delay (sec)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(os.path.join(output_folder, "flyover_4_hourly_comparison.png"))
plt.close()

# DAILY AVERAGE PLOT
def plot_daily(df):
    df["DATE"] = df["DATE&TIME"].dt.date
    return df.groupby("DATE")["DURATION_IN_TRAFFIC_SEC"].mean()

above_daily = plot_daily(above_df)
below_daily = plot_daily(below_df)

plt.figure(figsize=(14, 6))
plt.plot(above_daily.index, above_daily.values, label="Above Flyover", marker='o')
plt.plot(below_daily.index, below_daily.values, label="Below Flyover", marker='o')
plt.title("Flyover 4 - Day-wise Avg Traffic Delay (sec)")
plt.xlabel("Date")
plt.ylabel("Avg Delay (sec)")
plt.legend()
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.savefig(os.path.join(output_folder, "flyover_4_daily_comparison.png"))
plt.close()

# WEEKDAY-WISE AVERAGE PLOT
def plot_weekday(df):
    df["WEEKDAY"] = df["DATE&TIME"].dt.day_name()
    return df.groupby("WEEKDAY")["DURATION_IN_TRAFFIC_SEC"].mean()

# Define custom weekday order
weekday_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

above_weekday = plot_weekday(above_df).reindex(weekday_order)
below_weekday = plot_weekday(below_df).reindex(weekday_order)

plt.figure(figsize=(10, 6))
plt.plot(above_weekday.index, above_weekday.values, label="Above Flyover", marker='o')
plt.plot(below_weekday.index, below_weekday.values, label="Below Flyover", marker='o')
plt.title("Flyover 4 - Weekday-wise Avg Traffic Delay (sec)")
plt.xlabel("Day of Week")
plt.ylabel("Avg Delay (sec)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(os.path.join(output_folder, "flyover_4_weekday_comparison.png"))
plt.close()

print("All Flyover 4 plots generated and saved.")


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# === Setup ===
file_path = "forward_combined_cleaned.xlsx"  # Replace with your actual file
output_folder = "weekday_weekend_analysis"
os.makedirs(output_folder, exist_ok=True)

# Read all sheets
xls = pd.ExcelFile(file_path)
sheet_names = xls.sheet_names

all_data = []

# === Load and process each sheet ===
for sheet in sheet_names:
    df = xls.parse(sheet)
    
    # Normalize column names
    df.columns = [col.strip().lower() for col in df.columns]
    
    # Check for required columns
    if 'date' not in df.columns or 'traffic delay' not in df.columns:
        continue
    
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    df.dropna(subset=['date'], inplace=True)
    df['day_of_week'] = df['date'].dt.dayofweek
    df['is_weekend'] = df['day_of_week'] >= 5
    df['stretch'] = sheet
    all_data.append(df[['date', 'traffic delay', 'is_weekend', 'stretch']])

# === Combine all data ===
combined = pd.concat(all_data)

# === Grouped Statistics ===
grouped = combined.groupby('is_weekend')['traffic delay'].agg(['mean', 'std']).reset_index()
grouped['day_type'] = grouped['is_weekend'].map({False: 'Weekdays (Mon-Fri)', True: 'Weekends (Sat-Sun)'})

# === Bar Plot with Std Dev ===
plt.figure(figsize=(8, 6))
plt.bar(grouped['day_type'], grouped['mean'], yerr=grouped['std'], capsize=10, color=['skyblue', 'salmon'])
plt.ylabel('Average Traffic Delay (seconds)')
plt.title('Traffic Delay: Weekdays vs Weekends (with Variance)')
plt.tight_layout()

# Save bar plot
bar_path = os.path.join(output_folder, 'weekday_weekend_bar_variance.png')
plt.savefig(bar_path)
plt.close()
print(f"Bar chart saved to {bar_path}")

# === Box Plot for distribution ===
plt.figure(figsize=(8, 6))
combined['day_type'] = combined['is_weekend'].map({False: 'Weekdays (Mon-Fri)', True: 'Weekends (Sat-Sun)'})
combined.boxplot(column='traffic delay', by='day_type', grid=False)
plt.title('Traffic Delay Distribution: Weekdays vs Weekends')
plt.suptitle('')
plt.ylabel('Traffic Delay (seconds)')
plt.tight_layout()

# Save boxplot
box_path = os.path.join(output_folder, 'weekday_weekend_boxplot.png')
plt.savefig(box_path)
plt.close()
print(f"Box plot saved to {box_path}")


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# === Setup ===
file_path = "indexed_sheetwise_forward.xlsx"  # Your actual file name
output_folder = "weekday_weekend_analysis"
os.makedirs(output_folder, exist_ok=True)

# Read all sheets
xls = pd.ExcelFile(file_path)
sheet_names = xls.sheet_names

all_data = []

# === Load and process each sheet ===
for sheet in sheet_names:
    df = xls.parse(sheet)
    
    # Normalize column names
    df.columns = [col.strip().lower() for col in df.columns]
    
    # Check for required columns
    if 'date' not in df.columns or 'traffic delay' not in df.columns:
        continue
    
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    df.dropna(subset=['date'], inplace=True)
    df['day_of_week'] = df['date'].dt.dayofweek
    df['is_weekend'] = df['day_of_]()_]()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# === Setup ===
file_path = "indexed_sheetwise_forward.xlsx"  # Your actual file name
output_folder = "weekday_weekend_analysis"
os.makedirs(output_folder, exist_ok=True)

# Read all sheets
xls = pd.ExcelFile(file_path)
sheet_names = xls.sheet_names

all_data = []

# === Load and process each sheet ===
for sheet in sheet_names:
    df = xls.parse(sheet)
    
    # Normalize column names
    df.columns = [col.strip().lower() for col in df.columns]
    
    # Check for required columns
    if 'date' not in df.columns or 'traffic delay' not in df.columns:
        continue
    
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    df.dropna(subset=['date'], inplace=True)
    df['day_of_week'] = df['date'].dt.dayofweek
    df['is_weekend'] = df['day_of_week'] >= 5
    df['stretch'] = sheet
    all_data.append(df[['date', 'traffic delay', 'is_weekend', 'stretch']])

# === Combine all data ===
combined = pd.concat(all_data)

# === Grouped Statistics ===
grouped = combined.groupby('is_weekend')['traffic delay'].agg(['mean', 'std']).reset_index()
grouped['day_type'] = grouped['is_weekend'].map({False: 'Weekdays (Mon-Fri)', True: 'Weekends (Sat-Sun)'})

# === Bar Plot with Std Dev ===
plt.figure(figsize=(8, 6))
plt.bar(grouped['day_type'], grouped['mean'], yerr=grouped['std'], capsize=10, color=['skyblue', 'salmon'])
plt.ylabel('Average Traffic Delay (seconds)')
plt.title('Traffic Delay: Weekdays vs Weekends (with Variance)')
plt.tight_layout()

# Save bar plot
bar_path = os.path.join(output_folder, 'weekday_weekend_bar_variance.png')
plt.savefig(bar_path)
plt.close()
print(f"Bar chart saved to {bar_path}")

# === Box Plot for distribution ===
plt.figure(figsize=(8, 6))
combined['day_type'] = combined['is_weekend'].map({False: 'Weekdays (Mon-Fri)', True: 'Weekends (Sat-Sun)'})
combined.boxplot(column='traffic delay', by='day_type', grid=False)
plt.title('Traffic Delay Distribution: Weekdays vs Weekends')
plt.suptitle('')
plt.ylabel('Traffic Delay (seconds)')
plt.tight_layout()

# Save boxplot
box_path = os.path.join(output_folder, 'weekday_weekend_boxplot.png')
plt.savefig(box_path)
plt.close()
print(f"Box plot saved to {box_path}")


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# Load the Excel file
file_path = "indexed_sheetwise_forward.xlsx"
xls = pd.ExcelFile(file_path)

# Combine all data from sheets
all_data = []
for sheet in xls.sheet_names:
    df = xls.parse(sheet)
    df["DATE&TIME"] = pd.to_datetime(df["DATE&TIME"])
    df["Weekday"] = df["DATE&TIME"].dt.day_name()
    df["Is_Weekend"] = df["Weekday"].isin(["Saturday", "Sunday"])
    all_data.append(df)

# Combine into single DataFrame
combined_df = pd.concat(all_data, ignore_index=True)

# Group by Is_Weekend and compute average delay
avg_delay = combined_df.groupby("Is_Weekend")["Delay_Sec"].mean().reset_index()
avg_delay["Day_Type"] = avg_delay["Is_Weekend"].map({True: "Weekend", False: "Weekday"})

# Plotting
plt.figure(figsize=(8, 5))
plt.bar(avg_delay["Day_Type"], avg_delay["Delay_Sec"], color=["skyblue", "salmon"])
plt.title("Average Delay: Weekday vs Weekend")
plt.ylabel("Average Delay (seconds)")
plt.tight_layout()
plt.savefig("weekday_vs_weekend_delay.png")
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Setup
file_path = "indexed_sheetwise_forward.xlsx"
output_folder = "weekend_weekday_analysis"
os.makedirs(output_folder, exist_ok=True)

# Read all sheets
xls = pd.ExcelFile(file_path)
sheet_names = xls.sheet_names

all_data = []

for sheet in sheet_names:
    try:
        df = pd.read_excel(xls, sheet_name=sheet, usecols=["DATE&TIME", "Delay_Sec"])
        df.dropna(subset=["DATE&TIME", "Delay_Sec"], inplace=True)
        df["DATE&TIME"] = pd.to_datetime(df["DATE&TIME"], errors="coerce")
        df["Sheet"] = sheet
        df["Date"] = df["DATE&TIME"].dt.date
        df["Hour"] = df["DATE&TIME"].dt.hour
        df["Weekday_Name"] = df["DATE&TIME"].dt.day_name()
        df["Is_Weekend"] = df["Weekday_Name"].isin(["Saturday", "Sunday"])
        all_data.append(df)
    except Exception as e:
        print(f"Skipping {sheet}: {e}")

# Combine all data
combined_df = pd.concat(all_data, ignore_index=True)

# Plot 1: Average Delay - Weekday vs Weekend
avg_delay = combined_df.groupby("Is_Weekend")["Delay_Sec"].mean().reset_index()
avg_delay["Type"] = avg_delay["Is_Weekend"].map({True: "Weekend", False: "Weekday"})

plt.figure(figsize=(6, 5))
sns.barplot(x="Type", y="Delay_Sec", data=avg_delay, palette="Set2")
plt.title("Average Delay: Weekday vs Weekend")
plt.ylabel("Average Delay (sec)")
plt.tight_layout()
plt.savefig(os.path.join(output_folder, "avg_delay_weekday_vs_weekend.png"))
plt.show()

# Plot 2: Boxplot by Weekday
plt.figure(figsize=(10, 6))
sns.boxplot(x="Weekday_Name", y="Delay_Sec", data=combined_df,
            order=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"],
            palette="Set3")
plt.title("Delay Distribution by Weekday")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(os.path.join(output_folder, "boxplot_delay_by_weekday.png"))
plt.show()

# Plot 3: Standard Deviation by Weekday
std_dev = combined_df.groupby("Weekday_Name")["Delay_Sec"].std().reindex(
    ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
).reset_index()

plt.figure(figsize=(10, 5))
sns.barplot(x="Weekday_Name", y="Delay_Sec", data=std_dev, palette="coolwarm")
plt.title("Standard Deviation of Delay by Weekday")
plt.ylabel("Standard Deviation (sec)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(os.path.join(output_folder, "std_deviation_by_weekday.png"))
plt.show()

# Plot 4: Heatmap - Average Delay by Weekday & Hour
heatmap_data = combined_df.groupby(["Weekday_Name", "Hour"])["Delay_Sec"].mean().reset_index()
heatmap_pivot = heatmap_data.pivot(index="Weekday_Name", columns="Hour", values="Delay_Sec").reindex([
    "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"
])

plt.figure(figsize=(14, 6))
sns.heatmap(heatmap_pivot, cmap="YlOrRd", annot=True, fmt=".0f", linewidths=0.5)
plt.title("Average Delay (sec) by Weekday and Hour")
plt.xlabel("Hour of Day")
plt.ylabel("Weekday")
plt.tight_layout()
plt.savefig(os.path.join(output_folder, "heatmap_delay_by_weekday_hour.png"))
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Load Excel
file_path = 'indexed_sheetwise_forward.xlsx'
xls = pd.read_excel(file_path, sheet_name=None)

# Time bins
time_bins = [0, 3, 6, 9, 12, 15, 18, 21, 24]
bin_labels = ['12am-3am', '3am-6am', '6am-9am', '9am-12pm', '12pm-3pm', '3pm-6pm', '6pm-9pm', '9pm-12am']

# Create output directory
output_dir = 'traffic_plots'
os.makedirs(output_dir, exist_ok=True)

# Summary table
summary = pd.DataFrame()

# Process each sheet
for stretch_name, df in xls.items():
    if 'Time' not in df.columns or 'Speed_kmph' not in df.columns:
        continue

    df['Time'] = pd.to_datetime(df['Time'], errors='coerce')
    df = df.dropna(subset=['Time', 'Speed_kmph'])
    df['Hour'] = df['Time'].dt.hour
    df['Time_Bin'] = pd.cut(df['Hour'], bins=time_bins, labels=bin_labels, right=False, include_lowest=True)

    avg_speeds = df.groupby('Time_Bin')['Speed_kmph'].mean().reindex(bin_labels)
    avg_speeds.name = stretch_name
    summary = pd.concat([summary, avg_speeds], axis=1)

# Final table
summary = summary.transpose()
summary.index.name = 'Stretch'
summary = summary.astype(float)

# Save summary
summary.to_csv(os.path.join(output_dir, 'avg_speed_by_stretch_and_time.csv'))

# === 1. Heatmap ===
plt.figure(figsize=(12, 8))
sns.heatmap(summary, annot=True, cmap='YlOrRd')
plt.title('Average Speed (km/h) by Stretch and Time Bin')
plt.xlabel('Time of Day')
plt.ylabel('Road Stretch')
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'heatmap_avg_speed.png'), dpi=300)
plt.close()

# === 2. Line Plot ===
plt.figure(figsize=(14, 8))
for stretch in summary.index:
    plt.plot(summary.columns, summary.loc[stretch], label=stretch, alpha=0.5)
plt.title('Speed Variation Across Time Bins')
plt.xlabel('Time of Day')
plt.ylabel('Average Speed (km/h)')
plt.legend(loc='upper right', fontsize='small', ncol=2)
plt.grid(True)
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'lineplot_speed_variation.png'), dpi=300)
plt.close()

# === 3. Bar Chart ===
avg_by_stretch = summary.mean(axis=1).sort_values()
plt.figure(figsize=(12, 8))
sns.barplot(x=avg_by_stretch.values, y=avg_by_stretch.index, palette='coolwarm')
plt.title('Overall Average Speed by Stretch')
plt.xlabel('Average Speed (km/h)')
plt.ylabel('Stretch')
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'barchart_avg_speed.png'), dpi=300)
plt.close()

# === 4. Box Plot ===
plt.figure(figsize=(14, 6))
sns.boxplot(data=summary, orient='h', palette='Set3')
plt.title('Speed Distribution per Time Bin Across Stretches')
plt.xlabel('Average Speed (km/h)')
plt.ylabel('Time Bin')
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'boxplot_speed_distribution.png'), dpi=300)
plt.close()

# === 5. Identify Problematic Stretches ===
low_speed_flags = (summary < 20).sum(axis=1)
slow_stretches = low_speed_flags[low_speed_flags >= 4]
slow_stretches.to_csv(os.path.join(output_dir, "problematic_stretches.csv"), header=["LowSpeed_Bin_Count"])


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os

# Load Excel file
file_path = "indexed_sheetwise_forward.xlsx"
xls = pd.read_excel(file_path, sheet_name=None)

# Prepare output directory
output_dir = "traffic_plots"
os.makedirs(output_dir, exist_ok=True)

# Define time bins
time_bins = [0, 3, 6, 9, 12, 15, 18, 21, 24]
bin_labels = ['12am-3am', '3am-6am', '6am-9am', '9am-12pm',
              '12pm-3pm', '3pm-6pm', '6pm-9pm', '9pm-12am']

summary = pd.DataFrame()
skipped_sheets = []

# Process each sheet
for stretch_name, df in xls.items():
    if 'DATE&TIME' not in df.columns or 'Speed_kmph' not in df.columns:
        skipped_sheets.append(stretch_name)
        continue

    df = df.dropna(subset=['DATE&TIME', 'Speed_kmph'])
    df['DATE&TIME'] = pd.to_datetime(df['DATE&TIME'], errors='coerce')
    df = df.dropna(subset=['DATE&TIME'])
    df['Hour'] = df['DATE&TIME'].dt.hour
    df['Time_Bin'] = pd.cut(df['Hour'], bins=time_bins, labels=bin_labels, right=False, include_lowest=True)

    avg_speeds = df.groupby('Time_Bin')['Speed_kmph'].mean().reindex(bin_labels)
    avg_speeds.name = stretch_name
    summary = pd.concat([summary, avg_speeds], axis=1)

# Final formatting
summary = summary.transpose()
summary.index.name = 'Stretch'
summary = summary.astype(float)
summary.dropna(how='all', inplace=True)
summary.dropna(axis=1, how='all', inplace=True)

# Save summary table
summary.to_csv(os.path.join(output_dir, 'avg_speed_by_stretch_and_time.csv'))

# === 1. Heatmap ===
plt.figure(figsize=(12, 8))
sns.heatmap(summary, annot=True, cmap='YlOrRd')
plt.title('Average Speed (km/h) by Stretch and Time Bin')
plt.xlabel('Time of Day')
plt.ylabel('Road Stretch')
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'heatmap_avg_speed.png'), dpi=300)
plt.close()

# === 2. Line Plot ===
plt.figure(figsize=(14, 8))
for stretch in summary.index:
    plt.plot(summary.columns, summary.loc[stretch], label=stretch, alpha=0.5)
plt.title('Speed Variation Across Time Bins')
plt.xlabel('Time of Day')
plt.ylabel('Average Speed (km/h)')
plt.legend(loc='upper right', fontsize='small', ncol=2)
plt.grid(True)
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'lineplot_speed_variation.png'), dpi=300)
plt.close()

# === 3. Bar Chart: Overall Avg Speed by Stretch ===
avg_by_stretch = summary.mean(axis=1).sort_values()
plt.figure(figsize=(12, 8))
sns.barplot(x=avg_by_stretch.values, y=avg_by_stretch.index, palette='coolwarm')
plt.title('Overall Average Speed by Stretch')
plt.xlabel('Average Speed (km/h)')
plt.ylabel('Stretch')
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'barchart_avg_speed.png'), dpi=300)
plt.close()

# === 4. Box Plot: Speed Distribution by Time Bin ===
plt.figure(figsize=(14, 6))
sns.boxplot(data=summary, orient='h', palette='Set3')
plt.title('Speed Distribution per Time Bin Across Stretches')
plt.xlabel('Average Speed (km/h)')
plt.ylabel('Time Bin')
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'boxplot_speed_distribution.png'), dpi=300)
plt.close()

# === 5. Problematic Stretches ===
low_speed_flags = (summary < 20).sum(axis=1)
slow_stretches = low_speed_flags[low_speed_flags >= 4]
slow_stretches.to_csv(os.path.join(output_dir, "problematic_stretches.csv"), header=["LowSpeed_Bin_Count"])

# Optional: Print skipped sheets
print("Skipped sheets due to missing columns:", skipped_sheets)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# Load your data
df = pd.read_excel("your_file.xlsx", index_col=0)  # Assuming first column is Time Bin

# Create output directory
output_dir = "stretch_speed_plots"
os.makedirs(output_dir, exist_ok=True)

# Plot for each stretch
for stretch in df.columns:
    avg_speeds = df[stretch]
    mean_speed = avg_speeds.mean()
    std_speed = avg_speeds.std()

    plt.figure(figsize=(10, 5))
    plt.plot(df.index, avg_speeds, marker='o', label="Avg Speed", color='blue')
    plt.axhline(mean_speed, color='green', linestyle='--', label=f"Mean = {mean_speed:.2f}")
    plt.axhline(mean_speed + std_speed, color='red', linestyle=':', label=f"+1 Std Dev = {mean_speed + std_speed:.2f}")
    plt.axhline(mean_speed - std_speed, color='red', linestyle=':', label=f"-1 Std Dev = {mean_speed - std_speed:.2f}")

    plt.title(f"Speed Trend for {stretch}")
    plt.xlabel("Time Bin")
    plt.ylabel("Average Speed (km/h)")
    plt.xticks(rotation=45)
    plt.legend()
    plt.tight_layout()
    
    # Save the plot
    plt.savefig(f"{output_dir}/{stretch}_speed_trend.png")
    plt.close()

print(f"All plots saved to: {output_dir}")


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# Load your data
df = pd.read_excel("indexed_sheetwise_forward.xlsx", index_col=0) # Assuming first column is Time Bin

# Create output directory
output_dir = "stretch_speed_plots"
os.makedirs(output_dir, exist_ok=True)

# Plot for each stretch
for stretch in df.columns:
    avg_speeds = df[stretch]
    mean_speed = avg_speeds.mean()
    std_speed = avg_speeds.std()

    plt.figure(figsize=(10, 5))
    plt.plot(df.index, avg_speeds, marker='o', label="Avg Speed", color='blue')
    plt.axhline(mean_speed, color='green', linestyle='--', label=f"Mean = {mean_speed:.2f}")
    plt.axhline(mean_speed + std_speed, color='red', linestyle=':', label=f"+1 Std Dev = {mean_speed + std_speed:.2f}")
    plt.axhline(mean_speed - std_speed, color='red', linestyle=':', label=f"-1 Std Dev = {mean_speed - std_speed:.2f}")

    plt.title(f"Speed Trend for {stretch}")
    plt.xlabel("Time Bin")
    plt.ylabel("Average Speed (km/h)")
    plt.xticks(rotation=45)
    plt.legend()
    plt.tight_layout()
    
    # Save the plot
    plt.savefig(f"{output_dir}/{stretch}_speed_trend.png")
    plt.close()

print(f"All plots saved to: {output_dir}")


In [ ]:
# === 6. Individual Stretch Plots with Mean and Std Dev ===
individual_plot_dir = os.path.join(output_dir, "individual_stretch_plots")
os.makedirs(individual_plot_dir, exist_ok=True)

for stretch in summary.index:
    speeds = summary.loc[stretch]
    mean_speed = speeds.mean()
    std_speed = speeds.std()

    plt.figure(figsize=(10, 6))
    sns.lineplot(x=speeds.index, y=speeds.values, marker='o', label='Avg Speed')
    
    # Plot mean line
    plt.axhline(mean_speed, color='green', linestyle='--', label=f'Mean ({mean_speed:.2f} km/h)')
    
    # Plot shaded std dev area
    plt.fill_between(speeds.index,
                     mean_speed - std_speed,
                     mean_speed + std_speed,
                     color='green',
                     alpha=0.2,
                     label=f'±1 Std Dev ({std_speed:.2f})')

    plt.title(f'Average Speed Trend for {stretch}')
    plt.xlabel('Time Bin')
    plt.ylabel('Average Speed (km/h)')
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(individual_plot_dir, f"{stretch}_speed_plot.png"), dpi=300)
    plt.close()
